In [24]:


import jax.numpy as jnp
import numpy as np
def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)

In [25]:
x=np.array([0,0,0,0,1,1,1])
k=2

In [26]:
one_hot(x,k)

DeviceArray([[1., 0.],
             [1., 0.],
             [1., 0.],
             [1., 0.],
             [0., 1.],
             [0., 1.],
             [0., 1.]], dtype=float32)

In [27]:
import numpy as np
from pysat.formula import CNF
def do_it(instance_name):
        #cnf = CNF(from_file=(instance_name + ".cnf"))
        #print(cnf.clauses[0][0])

        target_name = instance_name + "_samples_sol.npy"
        target_func=np.load(target_name)


        #number_of_unsat=check_clauses(cnf.clauses,target_func)
        #print(number_of_unsat)
        #, target_func),axis=0)
        #print(violated_constraints_for_candidates)
        weights= 1#tbf    
        return((target_func,weights))

def assignment(x):
    if x > 0:
        return False
    elif x < 0:
        return True
    else:
        print("error, not a proper input")

def current_assignment(candidate, index):
        return candidate[index]

def check_clauses(clauses,candidate):
        count_unsat=0
        print(clauses[0][0])
        for i in range(len(clauses)):
                satisfied=0
                for j in range(len(clauses[i])):
                        if satisfied==0:
                                desired=assignment(clauses[i][j])
                                print("a",desired)
                                current= current_assignment(candidate[i],j)
                                print("b",current)
                                if desired==current:
                                        satisfied=1
                        if satisfied==0:
                                count_unsat+= 1
        return count_unsat

In [28]:
instance_name="/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/git_multiple_candidates/ml_based_sat_solver/Data/one_example/41-20486"

a=do_it(instance_name)

In [29]:
print(a)

(array([[False, False, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [False, False,  True, ..., False,  True, False],
       ...,
       [False, False,  True, ...,  True,  True,  True],
       [False,  True, False, ...,  True, False,  True],
       [False,  True,  True, ...,  True,  True,  True]]), 1)


In [30]:
import glob
import gzip
from os.path import join, exists, splitext
import pickle

import jraph
import nnf
from func_timeout import func_timeout, FunctionTimedOut
from torch.utils import data
import numpy as np
from pysat.formula import CNF
from collections import namedtuple
from constraint_problems import get_problem_from_cnf
from random_walk import violated_constraints
from jax import vmap


def create_candidates_with_sol(data_dir, sample_size, threshold):
    solved_instances = glob.glob(join(data_dir, "*_sol.pkl"))
    for g in solved_instances:
        with open(g, "rb") as f:
            p = pickle.load(f)
        n = np.array(list(p.values()), dtype=bool)
        samples = sample_candidates(n, sample_size-1, threshold)
        #print(samples)
        samples = np.concatenate((np.reshape(n,(1,len(n))),samples),axis=0)
        name = g.split('_sol.pkl')[0]
        with open(name + "_samples_sol.npy", "wb") as f:
            np.save(f, samples)
    #return(samples)



def sample_candidates(original, sample_size, threshold):
    np.random.seed(sum(original))
    condition = np.random.random((sample_size, original.shape[0])) < threshold
    return np.where(condition, np.invert(original), original)



In [31]:


data_dir="/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset"
sample_size=20
threshold=0.5

create_candidates_with_sol(data_dir, sample_size, threshold)

In [32]:
from random_walk import violated_constraints
import numpy as np

def number_of_violated_constraints(problem,candidate):
    return np.sum(violated_constraints(problem, candidate).astype(int),axis=0)

vio=number_of_violated_constraints(problem, candidates[0])
print(vio)

NameError: name 'problem' is not defined

In [ ]:
from jax import vmap
a=vmap(number_of_violated_constraints,in_axes=(None,0),out_axes=0)(problem,candidates)

In [ ]:
import time
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from torch.utils import data

from data_utils import SATTrainingDataset, JraphDataLoader
from model import network_definition    
    
path='/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset/'
sat= SATTrainingDataset(path)
print("loaded data sucessfully")




loaded data sucessfully


In [ ]:
problem=sat[0][0]
candidates=sat[0][1][0:2]
#weights=sat[2][2]
#print(weights)
#print(sat[0][0])
print(candidates)


new method


2022-12-13 14:29:32.919845: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2022-12-13 14:29:33.804350: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 1.884605s
Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necess

new method
[array([[False, False, False, ..., False, False, False],
       [False,  True, False, ..., False, False,  True],
       [False,  True,  True, ...,  True, False,  True],
       ...,
       [False,  True, False, ..., False,  True, False],
       [ True, False, False, ...,  True,  True,  True],
       [ True,  True,  True, ..., False, False,  True]]), DeviceArray([    1, 71330, 70546, 72344, 72796, 74300, 72189, 71263,
             72476, 72620, 72812, 72016, 72276, 71392, 73341, 72951,
             71868, 72936, 72676, 72900], dtype=int32)]


In [ ]:
print(candidates[1])

[    1 71330 70546 72344 72796 74300 72189 71263 72476 72620 72812 72016
 72276 71392 73341 72951 71868 72936 72676 72900]


In [1]:
import time
import optax
import haiku as hk
import jax.numpy as jnp
import jax
from torch.utils import data

from data_utils import SATTrainingDataset, JraphDataLoader
from model import network_definition

NUM_EPOCHS = 10
f=0.1

# # Make a batched version of the forwarding
# batched_predict = jax.vmap(network.apply, in_axes=(None, 0))


# def loss(params, problems, targets):
#     preds = batched_predict(params, problems)
#     return -jnp.mean(preds * targets)


def one_hot(x, k, dtype=jnp.float32):
    """Create a one-hot encoding of x of size k."""
    return jnp.array(x[:, None] == jnp.arange(k), dtype)


path='/Users/p403830/Library/CloudStorage/OneDrive-PorscheDigitalGmbH/programming/ml_based_sat_solver/BroadcastTestSet_subset/'
sat_data = SATTrainingDataset(path)
train_data, test_data = data.random_split(sat_data, [0.8, 0.2])

train_loader = JraphDataLoader(train_data, batch_size=4, shuffle=True)

network = hk.without_apply_rng(hk.transform(network_definition))
params = network.init(jax.random.PRNGKey(42), sat_data[0][0].graph)

opt_init, opt_update = optax.adam(1e-3)
opt_state = opt_init(params)

@jax.jit
def update(params, opt_state, x, y):
        #g = jax.grad(prediction_loss)(params, *x, y)

        #TBD!!! -> change this function here

        g=jax.grad(new_prediction_loss)(params, *x, y)

        ####

        updates, opt_state = opt_update(g, opt_state)
        return optax.apply_updates(params, updates), opt_state


@jax.jit
def prediction_loss(params, mask, graph, solution):
        decoded_nodes = network.apply(params, graph)
        solution = one_hot(solution, 2)
        # We interpret the decoded nodes as a pair of logits for each node.
        log_prob = jax.nn.log_softmax(decoded_nodes) * solution
        return -jnp.sum(log_prob * mask[:, None]) / jnp.sum(mask)


@jax.jit
def new_prediction_loss(params, graph, candidates, f: float):
            decoded_nodes = network.apply(params, graph)
            candidates = one_hot(candidates, 2)
            # We interpret the decoded nodes as a pair of logits for each node.
            log_prob = jax.nn.log_softmax(decoded_nodes) * candidates[0]
            weights = jax.nn.softmax(- f * candidates[1])
            weighted_log_probs = jnp.dot(log_prob, weights)
            return -weighted_log_probs
    
    

    

    # batched_loss = jax.vmap(prediction_loss, in_axes=(None, 0))


2022-12-13 16:41:18.780588: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2022-12-13 16:41:19.849534: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 2.068997s
Constant folding an instruction is taking > 1s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necess

In [ ]:
import numpy as np
print(np.shape(train_data))

2022-12-13 15:14:34.515162: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 2s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2022-12-13 15:14:34.608136: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 2.097126s
Constant folding an instruction is taking > 2s:

  slice.62 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necess

(14, 2)


/Applications/anaconda3/lib/python3.9/site-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [2]:
print(train_loader)


In [3]:
for (x,y) in train_loader:
    print(np.shape(x))
    print(np.shape(y))

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 18544 and the array at index 1 has size 12112

In [ ]:

print("Entering training loop")

for epoch in range(NUM_EPOCHS):
        start_time = time.time()
        for (x, y) in train_loader:
            params, opt_state = update(params, opt_state, x, y)
            # print(f"{batch} done")
        epoch_time = time.time() - start_time

        # train_acc = accuracy(params, train_images, train_labels)
        # test_acc = accuracy(params, test_images, test_labels)
        print("Epoch {} in {:0.2f} sec".format(epoch, epoch_time))

        #test_acc = jnp.mean(jnp.asarray([prediction_loss(params, p.mask, p.graph, s) for (p, s) in test_data]))

        #TBD!!!

        test_acc = jnp.mean(jnp.asarray([new_prediction_loss(p, p.graph, c, f) for (p, c) in test_data]))
        
        ##

        # print("Training set accuracy {}".format(train_acc))
        print("Test set accuracy {}".format(test_acc))

    # TODO: Save the model here


print("done")